In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from scipy.stats.stats import pearsonr
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.spatial.distance import pdist, squareform
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

In [2]:
songs = pd.read_csv("/Users/piyushkhemka/Desktop/mm-songs-db-tools-master/dfsongs.csv")
songs = songs.drop_duplicates()
songs = songs.reset_index()

In [3]:
songs.head()

,index,track_id,title,song_id,release,artist_id,artist_name,duration,artist_familiarity,artist_hotttnesss,...,key,key_confidence,loudness,mode,mode_confidence,song_hotttnesss,start_of_fade_out,tempo,time_signature,time_signature_confidence
0,0,TRALGCD128F42A6346,Do Ya Thing (Remix) (Explicit Album Version),SOOMUYV12A8C1409D7,Killa Season,ARMB95F1187B9B5B93,Cam'Ron,174.88934,0.858271,0.515325,...,2,0.626,-5.797,1,0.335,NaN,163.161,101.883,4,1.000
1,1,TRALGCL12903CE74A7,I Am Going (Album Version),SOFVEWB12AC4689917,Inspirational Journey,ARAPH9U1187B9B34F0,Randy Travis,228.75383,0.787878,0.543226,...,11,0.979,-11.366,1,0.812,NaN,218.639,99.141,4,0.313
2,2,TRALGCQ128F931D8D5,Ai De Ka Luo Li,SOXDYYS12AB0180A12,Aitai,ARVFGPJ1187B9A2477,Michelle & Vickie,194.21995,0.368404,0.387740,...,4,0.411,-4.991,0,0.334,NaN,187.339,120.014,3,0.461
3,3,TRALGEJ128F932552A,Canci�_n del Arco Iris,SOLSCUE12AB017D8AC,Chupi Guay - Canciones para jugar CD 1,AR99OWM11C8A4149FD,Arco Iris,199.96689,0.401848,0.315453,...,11,0.949,-9.132,1,0.738,NaN,189.852,105.157,4,0.889
4,4,TRALGFI128F93500EE,Belina,SOZJZQG12AB01828BD,X̩ povo,ARZEHMQ1187B9950C0,Paulo Flores,282.85342,0.465802,0.312848,...,4,0.100,-13.615,1,0.357,0.249066,278.616,101.767,3,0.931


In [4]:
songs["finalmode"] = songs["mode"] * songs["mode_confidence"]

In [5]:
songs["time_signature_final"] = songs["time_signature"] *songs["time_signature_confidence"]

In [6]:
songs["key_final"] = songs["key"] * songs["key_confidence"]

In [7]:
songs = songs.drop(["mode"],axis=1)
songs = songs.drop(["mode_confidence"],axis=1)
songs = songs.drop(["time_signature"],axis=1)
songs = songs.drop(["time_signature_confidence"],axis=1)
songs = songs.drop(["key"],axis=1)
songs = songs.drop(["key_confidence"],axis=1)
songs = songs.drop(["energy"],axis=1)
songs = songs.drop(["year"],axis=1)
songs.head()

,index,track_id,title,song_id,release,artist_id,artist_name,duration,artist_familiarity,artist_hotttnesss,end_of_fade_in,loudness,song_hotttnesss,start_of_fade_out,tempo,finalmode,time_signature_final,key_final
0,0,TRALGCD128F42A6346,Do Ya Thing (Remix) (Explicit Album Version),SOOMUYV12A8C1409D7,Killa Season,ARMB95F1187B9B5B93,Cam'Ron,174.88934,0.858271,0.515325,0.000,-5.797,NaN,163.161,101.883,0.335,4.000,1.252
1,1,TRALGCL12903CE74A7,I Am Going (Album Version),SOFVEWB12AC4689917,Inspirational Journey,ARAPH9U1187B9B34F0,Randy Travis,228.75383,0.787878,0.543226,0.528,-11.366,NaN,218.639,99.141,0.812,1.252,10.769
2,2,TRALGCQ128F931D8D5,Ai De Ka Luo Li,SOXDYYS12AB0180A12,Aitai,ARVFGPJ1187B9A2477,Michelle & Vickie,194.21995,0.368404,0.387740,0.000,-4.991,NaN,187.339,120.014,0.000,1.383,1.644
3,3,TRALGEJ128F932552A,Canci�_n del Arco Iris,SOLSCUE12AB017D8AC,Chupi Guay - Canciones para jugar CD 1,AR99OWM11C8A4149FD,Arco Iris,199.96689,0.401848,0.315453,0.000,-9.132,NaN,189.852,105.157,0.738,3.556,10.439
4,4,TRALGFI128F93500EE,Belina,SOZJZQG12AB01828BD,X̩ povo,ARZEHMQ1187B9950C0,Paulo Flores,282.85342,0.465802,0.312848,0.340,-13.615,0.249066,278.616,101.767,0.357,2.793,0.400


In [8]:
songs = songs.replace(to_replace=np.NaN,value=0.00)

In [9]:
songs.describe()

,index,duration,artist_familiarity,artist_hotttnesss,end_of_fade_in,loudness,song_hotttnesss,start_of_fade_out,tempo,finalmode,time_signature_final,key_final
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,9590.053700,238.507518,0.565230,0.385552,0.758616,-10.485668,0.193626,229.975465,122.915449,0.342694,1.965782,2.330447
std,5971.018945,114.137514,0.160528,0.143647,1.867952,5.399788,0.251809,112.195735,35.184412,0.278942,1.568063,2.310865
min,0.000000,1.044440,0.000000,0.000000,0.000000,-51.643000,0.000000,1.044000,0.000000,0.000000,0.000000,0.000000
25%,4770.750000,176.032200,0.467570,0.325266,0.000000,-13.163250,0.000000,168.856000,96.965750,0.000000,0.261000,0.338750
50%,9318.500000,223.059140,0.563584,0.380742,0.199000,-9.380000,0.000000,213.879000,120.161000,0.396000,2.007500,1.612500
75%,11818.250000,276.375060,0.668020,0.453858,0.421000,-6.532500,0.405116,266.292000,144.013250,0.569000,3.173000,3.850000
max,20962.000000,1819.767710,1.000000,1.082503,43.119000,0.566000,1.000000,1813.426000,262.828000,1.000000,7.000000,11.000000


In [10]:
q = songs
q = q.drop(["title"],axis=1)
q = q.drop(["song_id"],axis=1)
q = q.drop(["release"],axis=1)
q = q.drop(["artist_id"],axis=1)
q = q.drop(["artist_name"],axis=1)
q = q.set_index("track_id")



In [11]:
kmeans_model = KMeans(n_clusters = 15, random_state = 1)
y_pred=kmeans_model.fit_predict(q)
center= kmeans_model.cluster_centers_

In [12]:
kmeans_model

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=15, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=1, tol=0.0001,
    verbose=0)

In [13]:
q = np.array(q)

In [14]:
q

array([[  0.00000000e+00,   1.74889340e+02,   8.58270678e-01, ...,
          3.35000000e-01,   4.00000000e+00,   1.25200000e+00],
       [  1.00000000e+00,   2.28753830e+02,   7.87877770e-01, ...,
          8.12000000e-01,   1.25200000e+00,   1.07690000e+01],
       [  2.00000000e+00,   1.94219950e+02,   3.68404327e-01, ...,
          0.00000000e+00,   1.38300000e+00,   1.64400000e+00],
       ..., 
       [  2.09600000e+04,   1.93723630e+02,   4.33507626e-01, ...,
          5.65000000e-01,   8.20000000e-01,   9.31000000e-01],
       [  2.09610000e+04,   3.00825670e+02,   3.34456512e-01, ...,
          5.03000000e-01,   0.00000000e+00,   0.00000000e+00],
       [  2.09620000e+04,   2.09736690e+02,   6.09181992e-01, ...,
          0.00000000e+00,   1.26800000e+00,   6.30000000e-01]])

In [15]:
user = pd.read_csv("/Users/piyushkhemka/Downloads/train_triplets.txt",nrows = 100000)

In [16]:
tab = lambda x: pd.Series([i for i in x.split('\t')])
user = user['b80344d063b5ccb3212f76538f3d9e43d87dca9e	SOAKIMP12A8C130995	1'].apply(tab)

In [17]:
user = user.rename(index=str, columns={0: "userid", 1: "songid", 2 : "count"})

In [18]:
user.head()

,userid,songid,count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBNZDC12A6D4FC103,1


In [19]:
# Divide dataset into training data and testing dataset
# Training dataset is 80% and testing dataset is 20%

trainingdata = user.ix[:80000,:]
testingdata = user.ix[80000:100000,:]

In [20]:
trainingdata.head()

,userid,songid,count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBNZDC12A6D4FC103,1


In [21]:
testingdata.head()

,userid,songid,count
80000,d33cda518d1d2191ed75b4560da852771893958f,SOLUYHW12AB0186D2B,1
80001,d33cda518d1d2191ed75b4560da852771893958f,SOLXHDD12AB017F0FA,1
80002,d33cda518d1d2191ed75b4560da852771893958f,SOMBCSF12AB017F0FC,1
80003,d33cda518d1d2191ed75b4560da852771893958f,SOMLCMV12AB017EB5A,1
80004,d33cda518d1d2191ed75b4560da852771893958f,SOMPSKF12AB017F0E4,2


In [22]:
song = trainingdata.merge(songs, left_on='songid', right_on='song_id', how='inner')

In [23]:
test = song.groupby('userid').mean()

In [24]:
test

,index,duration,artist_familiarity,artist_hotttnesss,end_of_fade_in,loudness,song_hotttnesss,start_of_fade_out,tempo,finalmode,time_signature_final,key_final
userid,,,,,,,,,,,,
000ebc858861aca26bac9b49f650ed424cf882fc,7929.625000,260.175872,0.612228,0.460984,0.761750,-9.310000,0.464004,248.172750,125.601000,0.23425,1.254375,2.655625
0039bd8483d578997718cdc0bf6c7c88b679f488,5884.000000,206.771790,0.865022,0.916053,1.284500,-7.973500,0.796098,194.165500,128.159000,0.46350,2.270000,0.194000
006edf2afa5cba7e65ccc97892021a129d7012dd,40.000000,211.565260,0.877119,0.560913,0.177000,-1.810000,0.804750,211.565000,104.796000,0.00000,4.000000,2.004000
00a443baf550f4bbdd974ba73720abf2759166f3,5374.000000,209.344850,0.929030,0.750427,0.089000,-3.313000,0.760348,206.698000,120.010000,0.50300,4.000000,0.238000
01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,7732.500000,219.584855,0.396724,0.530312,2.569000,-4.861000,0.742584,206.791000,94.735500,0.57750,2.000000,4.350000
019d0d1c7a01f8736ba59a124160e5fc70666db7,8597.600000,189.993342,0.706937,0.467972,0.924800,-9.168600,0.446622,180.272800,112.569600,0.15040,2.440000,2.912200
02192554db8fe6d17b6309aabb2b7526a2e58534,5373.500000,206.450486,0.835536,0.654227,0.972300,-8.479600,0.542036,198.899400,132.406800,0.29110,3.072800,2.071400
02a3cd5161b9175d57f5033f18ab91d7b3e1f69b,10386.000000,381.178320,0.852639,0.535763,0.000000,-6.360000,0.693464,377.295000,134.500000,0.52800,1.908000,3.185000
03041e39e6f7994779855c780d04ff5f0afe1e1c,8223.500000,220.394645,0.867531,0.809005,0.149500,-4.587500,0.676970,210.770500,107.950500,0.00000,4.305000,2.310000


In [25]:
test = test.dropna()

In [26]:
allusers = test.index

In [27]:
reco = []
count = 0
for i in allusers:
    currentuser = []
    currentuser.append(i)
    thisuser = list(test.loc[i])
    neigh = NearestNeighbors(n_neighbors=10)
    neigh.fit(q)
    NearestNeighbors(algorithm='ball_tree', leaf_size=100,metric='euclidean')
    distance, indices = neigh.kneighbors([thisuser])
    indices=list(indices[0])
    for ind in indices:
        currentuser.append(songs.ix[ind]["song_id"])
    count += 1
    reco.append(currentuser)

In [28]:
recommendations = pd.DataFrame(data = reco, columns = ["user_id","song1","song2","song3","song4","song5","song6","song7","song8","song9","song10"])
recommendations

,user_id,song1,song2,song3,song4,song5,song6,song7,song8,song9,song10
0,000ebc858861aca26bac9b49f650ed424cf882fc,SOYMMRW12A8AE4625D,SONCTXN12A8C134A81,SOSUZKN12AB0182AED,SOKBGFX12AB0188810,SODSOBX12AC4686B4D,SOKPXAZ12A58A802CF,SOFLPJR12A8C134166,SOIQITW12AB018827D,SORVKAX12A8C1343AA,SOCQUSZ12A8C1421AE
1,0039bd8483d578997718cdc0bf6c7c88b679f488,SOMMALW12A58A79E93,SOUWYFC12AB0181DAD,SOGUDEQ12A6D4FAB25,SOGDSYD12AF72A0F02,SOSHGBG12A8151BD89,SOAIGNB12A67020200,SODRQHM12A8C1343AD,SOXTNNM12AB0182462,SONLHUA12A8C14470A,SODMJKG12A670202EB
2,006edf2afa5cba7e65ccc97892021a129d7012dd,SOAYOFO12AF72A4B88,SOWJALY12A6D4F837F,SOHHJYE12CF530E53A,SOCGBAY12AB017C78E,SOFJBSA12A8C130F97,SOPLCEU12AB018795B,SOQBIEB12AB0187D27,SOTLJUL12AF72A198D,SONSSFN12A58A7C412,SOMYIFM12AAF3B5E69
3,00a443baf550f4bbdd974ba73720abf2759166f3,SOIZLKI12A6D4F7B61,SOKFHLV12AB0187A2F,SOVGUDZ12AB017E644,SOTLKVX12A8C13BE9F,SOBBSDX12AAF3B33A9,SOHWHCE12A8C133385,SOJFVGM12A8C13E2B2,SOKTKHO12A58A7CDA5,SONPYGH12A6701F6CF,SOMZVHH12AB017D9B3
4,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,SOPCERW12AB018A2B5,SOHWAHE12A8C13DDD1,SOVGRXC12A6D4F94A8,SOIFDWL12A6D4F8A98,SOJCXNZ12A8C143C36,SOXLBKT12A8C14285E,SOFELMB12A8AE48525,SOIEAMX12A8C137AED,SOWOAZS12A8C14159D,SONDPLZ12A8C13270C
5,019d0d1c7a01f8736ba59a124160e5fc70666db7,SOXVLOJ12AB0189215,SOHKNRJ12A6701D1F8,SOFSOCN12A8C143F5D,SOMZWCG12A8C13C480,SORAMLE12AB017C8B0,SOCIWDW12A8C13D406,SONHOTT12A8C13493C,SOQHXMF12AB0182363,SOMUYGI12AB0188633,SOZVMJI12AB01808AF
6,02192554db8fe6d17b6309aabb2b7526a2e58534,SOIZLKI12A6D4F7B61,SOKFHLV12AB0187A2F,SOVGUDZ12AB017E644,SOMZVHH12AB017D9B3,SOJFVGM12A8C13E2B2,SOHWHCE12A8C133385,SONPYGH12A6701F6CF,SOTLKVX12A8C13BE9F,SOBBSDX12AAF3B33A9,SOKTKHO12A58A7CDA5
7,02a3cd5161b9175d57f5033f18ab91d7b3e1f69b,SOFKTPP12A8C1385CA,SOFXFXN12AB01827D6,SOMCPKY12AB0184197,SOGTVGQ12A8C13952B,SOLWWCO12A58A77015,SOCNCAD12AB0187375,SOFJEAL12A8C13E4C5,SORMDIB12AB018C316,SOBHZOV12AF729E943,SORYCLC12A582A293D
8,03041e39e6f7994779855c780d04ff5f0afe1e1c,SOYGZPA12AB0188EF2,SOLPZUJ12A81C21413,SOZVWSE12A6D4F7ADA,SODHQLP12A6D4FA6FA,SODQNJB12A6D4F66A0,SOQELHR12AC4689565,SOHZDWZ12A8C136FBD,SORGFZA12A8C13EF1E,SOYCABE12A8C142BD5,SOORYUV12AB0183C3E
9,037167e01a2b265b8ee59694db943f9556876be2,SOXOUJH12A6D4FC39B,SONLJKK12A8C1425F9,SOUMWKR12AB0181548,SOBBKHN12AC468BC73,SOVKTLW12A6310E32D,SOFJHBV12A6D4F446A,SOJUPRU12A58A7A5B6,SODGKWB12A8C13F612,SOVMXWO12AB0181DC9,SOGWIBP12AB017E424


In [29]:
keydict = {}
for user, song in zip(trainingdata.userid, trainingdata.songid):
    try:
        keydict[user].append(song)
    except KeyError:
        keydict[user] = []
        keydict[user].append(song)

In [30]:
reco = {}
for user, song1 in zip(recommendations.user_id, recommendations.song1):
    try:
        reco[user].append(song1)
    except KeyError:
        reco[user] = []
        reco[user].append(song1)
        
for user, song2 in zip(recommendations.user_id, recommendations.song2):
    try:
        reco[user].append(song2)
    except KeyError:
        reco[user] = []
        reco[user].append(song2)
        
for user, song3 in zip(recommendations.user_id, recommendations.song3):
    try:
        reco[user].append(song3)
    except KeyError:
        reco[user] = []
        reco[user].append(song3)
        
for user, song4 in zip(recommendations.user_id, recommendations.song4):
    try:
        reco[user].append(song4)
    except KeyError:
        reco[user] = []
        reco[user].append(song4)


for user, song5 in zip(recommendations.user_id, recommendations.song5):
    try:
        reco[user].append(song5)
    except KeyError:
        reco[user] = []
        reco[user].append(song5)
        
        
for user, song6 in zip(recommendations.user_id, recommendations.song6):
    try:
        reco[user].append(song6)
    except KeyError:
        reco[user] = []
        reco[user].append(song6)
        
for user, song7 in zip(recommendations.user_id, recommendations.song7):
    try:
        reco[user].append(song7)
    except KeyError:
        reco[user] = []
        reco[user].append(song7)
        
for user, song8 in zip(recommendations.user_id, recommendations.song8):
    try:
        reco[user].append(song8)
    except KeyError:
        reco[user] = []
        reco[user].append(song8)
        
for user, song9 in zip(recommendations.user_id, recommendations.song9):
    try:
        reco[user].append(song9)
    except KeyError:
        reco[user] = []
        reco[user].append(song9)
        
for user, song10 in zip(recommendations.user_id, recommendations.song10):
    try:
        reco[user].append(song10)
    except KeyError:
        reco[user] = []
        reco[user].append(song10)

In [31]:
match = 0
nonmatch = 0
count = 0
for i in reco:
    recoset = set(reco[i])
    if i in keydict:
        keydictset = set(keydict[i])
        matches = (recoset - (recoset - keydictset))
        match += len(matches)
        nonmatch += 10 - len(matches)

In [32]:
match = float(match)
nonmatch = float(nonmatch)

In [33]:
totalreco = recommendations.shape[0]*(recommendations.shape[1]-1)

In [34]:
totalreco

6760

In [35]:
precision = float(match/(match+nonmatch))

In [36]:
precision = precision*100

In [62]:
songs_sample = songs.set_index("song_id")

In [63]:
mylist = []
for i in range(1,len(recommendations.ix[0])):
    mylist.append(songs_sample.ix[recommendations.ix[0][i]]["title"])
    
samplerecommendation = pd.DataFrame(data = mylist)
samplerecommendation = samplerecommendation.rename(index=str, columns={0: recommendations.ix[0][0]})

In [64]:
samplerecommendation

,000ebc858861aca26bac9b49f650ed424cf882fc
0,Genio Atrapado
1,Did We Not Choose Each Other
2,So So So
3,Life Deprived
4,Warhead (Live in Croatia_ 1993)
5,Baltech's Lament
6,Saturday
7,Take Your Leave Of Me Baby
8,Man I Used To Be
9,The west's awake
